## Loading data and brief description data.

We start by importing packages we need and loading the data. We will use pandas to store the data. This structure modeled on Worksheet 3 and the scripts from Wikitalk.

In [1]:
# Import necessary packages
import pandas as pd
import datetime
import numpy as np
import re
import sys
import random
import math
from nltk import FreqDist, ngrams, sent_tokenize, word_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from sklearn import svm
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import accuracy_score

# Load the training data. Scramble the rows (sometimes this is important for training)
df = pd.read_csv("python_data/train",sep="\t",error_bad_lines=False,encoding="utf-8",nrows=50000)
df = df.sample(frac=1)

# Load some datasets we will need later on. E.g. English stopwords.
eng_stopwords = stopwords.words('english') 

Let us first take a brief look at the data. We see that there are approximately 323K comments in the training data. For each comment, we have the native language of the writer (native_lang), the self-reported level of the english speaker (native, 1, 2, 3, 4, 5 or unknown), the original text of the comment (text_original), the text with demonyms and proper nouns replaced by PoS tags (text_clean), and finally the structure of the text reported by SENNA (text_structure). Of the 323K comments, about 110K comments are from native speakers. Thus, the training data seems sufficiently balanced.

In [2]:
df.describe()

,native_lang,text_original,level_english,text_clean,text_structure
count,50000,50000,50000,50000,50000
unique,20,49603,7,49600,49546
top,EN,is being used on this article. I notice the im...,N,is being used on this article. I notice the im...,VBZ VBG VBN IN DT NN . PRP VBP DT NN NN VBZ IN...
freq,17172,109,17172,109,109


In [3]:
df.native_lang.describe()

count     50000
unique       20
top          EN
freq      17172
Name: native_lang, dtype: object

In [4]:
df['native'] = np.where(df['native_lang']=='EN', "native", "non-native")
df.describe()

,native_lang,text_original,level_english,text_clean,text_structure,native
count,50000,50000,50000,50000,50000,50000
unique,20,49603,7,49600,49546,2
top,EN,is being used on this article. I notice the im...,N,is being used on this article. I notice the im...,VBZ VBG VBN IN DT NN . PRP VBP DT NN NN VBZ IN...,non-native
freq,17172,109,17172,109,109,32828


## Replication Al-'Rfou

The paper on which we've based our project on uses similarity scores to word and character n-gram models as the features for subsequent classification. Let us embark too on construction of such models. However, other literature has shown that for character n-grams, increasing n seems to enhance classifcation. Thus, we will construct models for up to 10 n-gram models.

We can start by tokenizing sentences.

In [5]:
#df['tokenized_sents'] = df.apply(lambda row: sent_tokenize(row['text_clean']), axis=1)
#df['tokenized_struc'] = df.apply(lambda row: sent_tokenize(row['text_structure']), axis=1)

Now we derive distributions of language use of n-grams among native and non-native speakers.

In [6]:
def language_distribution(n, m, lowerfreqlimit, training, LANGUAGES):
    """Calculate the word n grams distribution up to n, the character n gram distribution up to m.
    @n: consider k-grams up to and including n for words, part of speech tags and word sizes.
    @m: consider k-grams up to and including m for characters.
    @lowerfreqlimit: number below which we consider words misspellings, odd words out or unique.
    @training: training data to retrieve the language distribution from.
    @LANGUAGES: languages based on which we classify.
    """
    
    language_dist = {}

    for language in LANGUAGES:
        language_dist[language] = {"words": dict(zip(range(1, n+1), [FreqDist() for i in range(1, n+1)])),
                               "tags": dict(zip(range(1, n+1), [FreqDist() for i in range(1, n+1)])),
                               "chars": dict(zip(range(1, m+1), [FreqDist() for i in range(1, m+1)])),
                               "w_sizes": dict(zip(range(1, n+1), [FreqDist() for i in range(1, n+1)]))}
    
    # First fill up 1-grams for words. This is necessary since we should ignore unique words for
    # higher order n-grams or we will get combinatorial explosions with result in memory errors.
    
    for language, tokenized_sents, tokenized_struc in training.itertuples(index=False):
        for sentence in tokenized_sents:
            token=word_tokenize(sentence)
            wordlens = [len(word) for word in token]  
            language_dist[language]["words"][1].update(ngrams(token,1))
            language_dist[language]["w_sizes"][1].update(ngrams(wordlens,1))
            
    # Now do it for all higher order k-grams, ignoring word k-grams that occur at most 1 time.
       
    unique_words = set(filter(lambda x: language_dist["non-native"]["words"][1][x] > lowerfreqlimit, language_dist["non-native"]["words"][1]))
    
    for language, tokenized_sents, tokenized_struc in training.itertuples(index=False):
    
        # Construct n grams counts from the tokenized sentences.
        for sentence in tokenized_sents:
            token = word_tokenize(sentence)     
            wordlens = [len(word) for word in token if word.isalpha()]   
                        
            # Start from index 2 since 1 is already filled.
            for k in range(2,n+1):  
                
                # Allow all word length k-grams.
                language_dist[language]["w_sizes"][k].update(ngrams(wordlens,k))
                
                # Remove any ngrams that have words which are unique.
                allowed_ngrams=[]                
                for ngram in ngrams(token,k):
                    allowed = True
                    for word in ngram:
                        if word in unique_words:
                            allowed=False 
                    if allowed:
                        allowed_ngrams.append(ngram)              
                
                #allowed_ngrams = [ngram for ngram in ngrams(token,k) if all(word not in unique_words for word in ngram)]
                
                language_dist[language]["words"][k].update(FreqDist(allowed_ngrams))             
                
                
        # Construct n gram counts from sentences tokenized based on structure.
        for sentence in tokenized_struc:
            token = word_tokenize(sentence)
            for k in range(1,n+1):
                language_dist[language]["tags"][k].update(ngrams(token,k))

        # Construct character m-grams for tokenized sentences.
        for sentence in tokenized_sents:
            for k in range(1,m+1):
                language_dist[language]["chars"][k].update(ngrams(sentence,k))
    
    return language_dist

With the distribution of grams over different languages, we can compute similarity scores. Note, for each language one can compute similarity score against each k-gram model. Thus, this results here in 2*(4+4+4+10) = 2x22 = 44 features. The paper mentiones that the scores are calculated as the sum of the 2-logs of counts in the model. Inspection of the scripts, however, show that this is not the case: this 2-log is normalized by the length of the sequence.

In [7]:
def compute_similarity_score(dis_ngramdic, gramlist):
    """ This function computes the similarity scores for a comment based on the corresponding k-grams.
    Note that the comment is already tokenized into sentences.
    @dis_ngramdic: ngram dictionary as constructed by language_distribution for particular k.
    @gramlist: list of kgrams
    """
    score=0
    if gramlist:
        for gram in gramlist:
            score += math.log2(dis_ngramdic.get(gram,1))
    return score

colnames = None

def compute_all_features(lang_dis, tokenized_sent, tokenized_struc):
    """ This function compares the tokenized sentences and tokenized structure to each of the languages distributions.
    It returns similarity scores to each language model. Also included are other features, such as the number of sentences
    per 
    @lang_dis: Language distribution of n-grams.
    @tokenized_sents: sentences tokenized by nltk.ngrams
    @tokenized_struc: PoS structure retrieved by SENNA, tokenized by nltk.ngrams.``
    """
    simscoredict=dict()

    # For each gramtype, first construct the list of which we can make n-grams.
    wordlens_ps = []
    words_ps = []
    struc_ps = []
    
    for sentence in tokenized_sent:
        wordlens_ps.append([len(word) for word in word_tokenize(sentence) if word.isalpha()])
        words_ps.append([word for word in word_tokenize(sentence)])
    
    for sentence in tokenized_struc:
        struc_ps.append([word for word in word_tokenize(sentence)])
    
    # Now we should construct k-gram lists for each k and return the score. Let us store all grams in 
    for gramtype in lang_dis[list(lang_dis.keys())[0]].keys():
        
        # Select appropriate data type.
        if gramtype == "tags":
            ps = struc_ps
        elif gramtype =="words":
            ps = words_ps
        elif gramtype == "w_sizes":
            ps = wordlens_ps
        else:
            ps = tokenized_struc
            
        seq_len = sum([len(item) for item in ps])
            
        # Construct for each k a gramlist. 
        for k in range(1,len(lang_dis[list(lang_dis.keys())[0]][gramtype])+1):
            
            kgramlist = list(set([gram for sentence in ps for gram in ngrams(sentence, k)]))
            
            for lang in lang_dis.keys():
                simscoredict[lang+'_'+gramtype+'_'+str(k)]= compute_similarity_score(lang_dis[lang][gramtype][k], kgramlist)/seq_len 
        
    # Set the other features they use in the paper.
    simscoredict["num_sentences"] = len(tokenized_sent) if isinstance(tokenized_sent, list) else 0
    simscoredict["num_words"] = sum([len(wc) for wc in wordlens_ps])
    simscoredict["avg_wordlength"] = sum([sum(word) for word in wordlens_ps])/simscoredict["num_words"]
    
    global colnames
    if colnames == None:
        colnames = list(simscoredict.keys())
            
    return simscoredict.values()

Al Rfou reports having used some different features too. It is not entirely clear what they mean. These include:
- "Relative frequency of each of the stop words mentioned in the comment"
- "Average number of sentences"
- "Size of the comments"

What these mean is not unequivocally clear. How should relative frequency be measured? Each comment has a deterministic number of sentences, so the average of sentences over what? The size of the comments, measured in what way? Since such features are ambiguous in their definition and are not reported to be important for the native vs non-native experiment, we exclude them.

In addition to the problems above, the paper does not detail how models were constructed. It mentions that approximately 322K features were used in the experiment and the baseline is 1/(number of classes). The first number makes sense as we have approximately 323K features in total. However, about 110K of these are native US English, whereas the other 200K are non-native speakers. It is not mentioned whether the non-native comments should be downsampled such that we have a balanced problem. We will assume this is the case.

### Repetition of the non-native experiment using SVM classification

Note, we cannot use the cross-validation function from `sklearn`. The reason for this is that the language model distribution has to be reconstructed for each fold so that there is no dependency between training and validation data. Let us therefore do it explicitly.

In [8]:
# Downsample the non-native languages so that the classes are balanced.
randomsample = pd.concat([df[df.native == "non-native"].sample(sum(df.native == "native")), df[df.native=="native"]])
randomsample = randomsample.sample(frac=1)
randomsample.native = randomsample.native.astype('category')

# Parameters to be used for training
n = 4    # n-grams for words, PoS tags and word sizes.
m = 9    # m-grams for characters
k = 5    # Number of folds for cross-validation

# Generator for folds. Note data has already been shuffled. Doing it won't hurt anyone though.
kf = KFold (n_splits = k, shuffle = True, random_state = k)

scores = []

for split in kf.split(randomsample):
        
    # Training and validation data.
    training = randomsample.iloc[split[0]]
    validation = randomsample.iloc[split[1]]
    
    # Tokenize sentences and structures for training data
    training['tokenized_sents'] = training.apply(lambda row: sent_tokenize(row['text_clean']), axis=1)
    training['tokenized_struc'] = training.apply(lambda row: sent_tokenize(row['text_structure']), axis=1)
    validation['tokenized_sents'] = validation.apply(lambda row: sent_tokenize(row['text_clean']), axis=1)
    validation['tokenized_struc'] = validation.apply(lambda row: sent_tokenize(row['text_structure']), axis=1)
    
    # Derive the language distribution based on training data.
    lang_dis = language_distribution(n,m,2,training[['native','tokenized_sents','tokenized_struc']], training.native.unique())
    
    # Use the language distribution to obtain features for training data.
    features = training.apply(lambda row: compute_all_features(lang_dis,row['tokenized_sents'], row['tokenized_struc']), axis=1)
    features = pd.DataFrame(features.to_frame()[0].values.tolist(), index=features.to_frame()[0].index, columns=colnames)
    training = pd.merge(training, features, left_index=True, right_index=True)
    
    # Use the language distribution to compute similarity data for validation data.
    features = validation.apply(lambda row: compute_all_features(lang_dis,row['tokenized_sents'], row['tokenized_struc']), axis=1)
    features = pd.DataFrame(features.to_frame()[0].values.tolist(), index=features.to_frame()[0].index, columns=colnames)
    validation = pd.merge(validation, features, left_index=True, right_index=True)
    
    # Train linear SVM classifier.
    linear = svm.SVC(kernel='linear')
    linear.fit(training[colnames], training.native)
    y_predicted = linear.predict(validation[colnames])
    scores.append(accuracy_score(validation.native, y_predicted))

print(scores)

/home/hans/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/hans/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/hans/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/in

[0.67011209783083414, 0.67025767942932013, 0.67375163779298297, 0.66792837385354487, 0.67821782178217827]


## Problems
- The language model distribution is based on all training samples. This actually means that our features already know some information about the class labels in cross-validation, which is kind of prohibited. Fixed, but maybe not the most efficient implementation as it has to reconstruct the model k times.
- Warnings. Should be able to suppress them. Not a problem for running but ugly.

In [9]:
#df2 = df.apply(lambda row: compute_all_features(lang_dis,row['tokenized_sents'], row['tokenized_struc']), axis=1)
#df2 = pd.DataFrame(df2.to_frame()[0].values.tolist(), index=df2.to_frame()[0].index, columns=colnames)
#df = pd.merge(df, df2, left_index=True, right_index=True)